In [1]:
from google.colab import files
uploaded = files.upload()

Saving Eugeniy_Onegin.txt to Eugeniy_Onegin.txt


In [2]:
import numpy as np
import tensorflow as tf
import os

In [3]:
with open('/content/Eugeniy_Onegin.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [4]:
len(text)

286984

In [5]:
print(text[:400])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        


In [6]:
text = text + text

In [7]:
len(text)

573968

In [8]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [9]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
"".join([idx2char[i] for i in text_as_int[:26]])

'Александр Сергеевич Пушкин'

In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(26):
    print(idx2char[i.numpy()], end="")

Александр Сергеевич Пушкин

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [15]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int64, name=None), TensorSpec(shape=(128, 100), dtype=tf.int64, name=None))>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [17]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [18]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [19]:
def generate_text(model, start_string, temperature):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [43]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [44]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru (GRU)                   (None, None, 1024)        3545088   
                                                                 
 dense_1 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 3696131 (14.10 MB)
Trainable params: 3696131 (14.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
model.compile(optimizer='adam', loss=loss)

In [47]:
tf.device('CUDA_VISIBLE_DEVICES')

In [51]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
44/44 [==============================] - 5s 97ms/step - loss: 0.1658
Epoch 2/100
44/44 [==============================] - 5s 98ms/step - loss: 0.1454
Epoch 3/100
44/44 [==============================] - 6s 99ms/step - loss: 0.1200
Epoch 4/100
44/44 [==============================] - 5s 100ms/step - loss: 0.1043
Epoch 5/100
44/44 [==============================] - 6s 101ms/step - loss: 0.0962
Epoch 6/100
44/44 [==============================] - 6s 105ms/step - loss: 0.1176
Epoch 7/100
44/44 [==============================] - 6s 102ms/step - loss: 0.0972
Epoch 8/100
44/44 [==============================] - 5s 102ms/step - loss: 0.0917
Epoch 9/100
44/44 [==============================] - 6s 100ms/step - loss: 0.0896
Epoch 10/100
44/44 [==============================] - 5s 100ms/step - loss: 0.0886
Epoch 11/100
44/44 [==============================] - 5s 102ms/step - loss: 0.0879
Epoch 12/100
44/44 [==============================] - 5s 102ms/step - loss: 0.0876
Epoch 13/100
44/

In [64]:
text_ = generate_text(model, 'Вниманье дружбы возлюбя', 1.7)

In [65]:
print(text_)

Вниманье дружбы возлюбя,
 Улегнень   ПрогЛАпрехот  гря.
  ику  Вотнисуволетсмышеведиъьешнамисв ".
 почьяничтуздян.
  г  гопосмычей;
   м  бщузДай V
 чтвсваныепабыла дыенишеX

 исанящаца.
 луж идегратсе;
  аяжазат,  wXОг.
  сестижимег!
 Гл.  Кашенг Оль. гимо ".
 щаку).
 ляхунущегид;
 любыворен. приньгедуморидясуя  грьсь.
 У СплXX

 уСт". 
 Пупи  Шиwera..
  ТвслНумопланаже; БомуВадам..

 Ужакойсичая,
 -
  обы,
 блВя. ялосудных
 чул;
  XXLI

 ..

  Пит.
 ци,
 бey ГЛечека, молые Блы,
 янижая иХL..   Ей?
   втнь пл7.
 "Ва 
